In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import copy
sw = set(stopwords.words('english'))
import pandas as pd
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/placid_brain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/placid_brain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
df1 = pd.read_csv("/Users/placid_brain/Documents/IR stuff/lsa/recipes_w_search_terms.csv",nrows=1000)

def function(ini_list):
    new_cell = ini_list.strip('][').split(', ')
    for item in new_cell:
        item = item.replace("'","")
    return new_cell

df1['ingredients'] = df1['ingredients'].apply(function)


df2 = df1[['id', 'name','ingredients','steps']]
#print(df2.head())

ingredients = df2['ingredients'].tolist()

new_lst =  [' '.join(i) for i in ingredients]

tfvec = TfidfVectorizer()
# turning ingredients-dishes(doc id) into tf-df vector
tdf = tfvec.fit_transform(new_lst).T
# singular value decomposition
[dc,cs,tc] = np.linalg.svd(tdf.toarray(), full_matrices=False)




In [18]:
class LSA:
    def __init__(self):
        pass
    def query_vector_gen(self,q_list):
        # turning query into tf-idf vector
        q = tfvec.transform(q_list)
        q = q.toarray()
        # mapping query vector to document-concept matrix and eventually concept strength matrix
        q_dc = np.matmul(q,dc)
        q_dc__s = np.matmul(q_dc,np.linalg.inv(np.diag(cs)))
        return q_dc__s
    
    def search(self, query):
        query = query.lower()
        
        query_tokens = word_tokenize(query)
        q_list = []

        for w in query_tokens:
            if w not in sw:
                q_list.append(w)
        res = " ".join(q_list)
        final_list =[]
        final_list.append(res)
        

        result = self.query_vector_gen(final_list)
        
        lst = []

        for arr in range(len(tc)):
            # mapping resultant vector to term-concept matrix
            lst.append(np.dot(result,tc[:,arr]))
        
        temp_lst = copy.deepcopy(lst)
        desc_lst = sorted(lst,reverse=True)[0:5]
        for i in range(len(desc_lst)):
            
            print("{}. {}".format(i+1, df2["name"][temp_lst.index(desc_lst[i])]))

  

In [19]:

lsa = LSA()
start = time.time()
print(lsa.search('sAlt crEAm and rice and buTter water ice lemon pepper'))
end = time.time()

1. Lemon Dressing
2. Christine's Super Chocolate Cake
3. Blueberry Waffles
4. Almond Kulich
5. Mom Hart's Pie Crust
None


In [20]:
print("Time taken: ", end-start)

Time taken:  0.08963418006896973
